In [ ]:
## Download or import the data
data_path = "/content/modcloth_final_data.json"

*Sonnv*

Tôi dự định thực hiện bài này như một nhiệm vụ Phân tích Dữ liệu Khám phá (Exploratory Data Analysis - EDA) đơn giản nhưng kỹ lưỡng bằng Python, seaborn và matplotlib.

### Mục lục:

1. [Về bộ dữ liệu](#1)
2. [Nhập dữ liệu - Modcloth](#3)
3. [EDA & Tiền xử lý](#4)
   1. [Biểu đồ hộp (Boxplot) của các biến số](#5)
   2. [Xử lý các giá trị ngoại lai (Outliers)](#6)
   3. [Phân phối kết hợp của bra_size và size](#7)
4. [Làm sạch & Tiền xử lý dữ liệu](#8)
   1. [Phân phối ban đầu của các đặc trưng](#9)
   2. [Xử lý đặc trưng từng bước](#10)
      1. [Kỹ thuật đặc trưng (Feature Engineering) - thêm đặc trưng mới](#11)
5. [EDA thông qua Trực quan hóa](#12)
   1. [Phân phối của các đặc trưng](#13)
   2. [Các danh mục (Categories) so với Độ vừa vặn/Chiều dài/Chất lượng (Fit/Length/Quality)](#14)
   3. [Người dùng (Users) so với Số lượng mặt hàng đã mua (Items bought)](#15)
   4. [Chiều cao (Height) so với Cỡ giày (Shoe-size)](#16)
6. [Tài liệu tham khảo](#references)
7. [Các giả định](#assumptions)


<a id="1"></a>
### Về bộ dữ liệu

> Bộ dữ liệu này chứa phản hồi về độ vừa vặn của quần áo do khách hàng tự báo cáo cũng như các thông tin phụ khác như đánh giá, xếp hạng, danh mục sản phẩm, kích cỡ theo danh mục, số đo của khách hàng (v.v.) từ 2 trang web:
>
> 1. [Modcloth](http://modcloth.com)
> 2. [Renttherunway](http://renttherunway.com)
>
> [[1]](#references) ModCloth bán quần áo và phụ kiện cổ điển dành cho phụ nữ, từ đó người quản lý bộ dữ liệu đã thu thập dữ liệu từ ba danh mục: váy, áo và quần. RentTheRunWay là một nền tảng độc đáo cho phép phụ nữ thuê quần áo cho các dịp khác nhau; họ đã thu thập dữ liệu từ một số danh mục.
>
> **Lưu ý:** Trong cả hai bộ dữ liệu, phản hồi về độ vừa vặn thuộc một trong ba loại: ‘Nhỏ’ (Small), ‘Vừa’ (Fit) và ‘Lớn’ (Large). Và cũng có một số [giả định](#assumptions) đã được đưa ra về các đặc trưng trong bộ dữ liệu.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Suppressing all warnings
import warnings
warnings.filterwarnings("ignore")

import matplotlib
matplotlib.rc('figure', figsize = (20, 8))
matplotlib.rc('font', size = 14)
matplotlib.rc('axes.spines', top = False, right = False)
matplotlib.rc('axes', grid = False)
matplotlib.rc('axes', facecolor = 'white')

<a id="2"></a>
# Bộ dữ liệu [Modcloth](http://modcloth.com)
<a id="3"></a>
## Nhập dữ liệu bằng Pandas

Xem qua một vài dòng đầu tiên của tệp json dữ liệu modcloth bằng lệnh bash tích hợp `head` của hệ điều hành.

In [ ]:
# Execute this in your kernel to view the first n (here-4) lines of the json file.
! head -n 4 /content/modcloth_final_data.json

{"item_id": "123373", "waist": "29", "size": 7, "quality": 5, "cup size": "d", "hips": "38", "bra size": "34", "category": "new", "bust": "36", "height": "5ft 6in", "user_name": "Emily", "length": "just right", "fit": "small", "user_id": "991571"}
{"item_id": "123373", "waist": "31", "size": 13, "quality": 3, "cup size": "b", "hips": "30", "bra size": "36", "category": "new", "length": "just right", "height": "5ft 2in", "user_name": "sydneybraden2001", "fit": "small", "user_id": "587883"}
{"item_id": "123373", "waist": "30", "size": 7, "quality": 2, "cup size": "b", "shoe size": "9.00", "bra size": "32", "category": "new", "length": "slightly long", "height": "5ft 7in", "user_name": "Ugggh", "fit": "small", "user_id": "395665"}
{"item_id": "123373", "category": "new", "size": 21, "quality": 5, "user_name": "alexmeyer626", "length": "just right", "fit": "fit", "cup size": "dd/e", "user_id": "875643"}


Sử dụng hàm `pd.read_json()`, tệp json được đưa vào một DataFrame của pandas, với tham số `lines` là `True` - bởi vì mỗi đối tượng mới được phân tách bằng một dòng mới.

In [ ]:
# Code Here

df_raw = pd.read_json(data_path,lines = True )
df_raw.head()


,item_id,waist,size,quality,cup size,hips,bra size,category,bust,height,user_name,length,fit,user_id,shoe size,shoe width,review_summary,review_text
0,123373,29.0,7,5.0,d,38.0,34.0,new,36,5ft 6in,Emily,just right,small,991571,NaN,NaN,NaN,NaN
1,123373,31.0,13,3.0,b,30.0,36.0,new,NaN,5ft 2in,sydneybraden2001,just right,small,587883,NaN,NaN,NaN,NaN
2,123373,30.0,7,2.0,b,NaN,32.0,new,NaN,5ft 7in,Ugggh,slightly long,small,395665,9.0,NaN,NaN,NaN
3,123373,NaN,21,5.0,dd/e,NaN,NaN,new,NaN,NaN,alexmeyer626,just right,fit,875643,NaN,NaN,NaN,NaN
4,123373,NaN,18,5.0,b,NaN,36.0,new,NaN,5ft 2in,dberrones1,slightly long,small,944840,NaN,NaN,NaN,NaN


<a id="4"></a>
# EDA - Phân tích Dữ liệu Khám phá

Chúng ta đã có thể đưa ra một vài quan sát ở đây, bằng cách nhìn vào phần đầu của dữ liệu:

1. Có các giá trị bị thiếu (missing values) trong DataFrame, cần phải được xử lý.
2. Cột `cup_size` chứa nhiều sở thích khác nhau - sẽ cần xử lý nếu chúng ta muốn định nghĩa `cup_size` là kiểu dữ liệu 'category'.
3. Cột `height` cần được phân tích cú pháp để trích xuất chiều cao dưới dạng một đại lượng số, hiện tại nó trông giống như một chuỗi (object).
4. Không quá quan trọng, nhưng một số cột có thể cần đổi tên - để loại bỏ khoảng trắng.

Đầu tiên, chúng ta xử lý việc đặt tên các cột để dễ dàng truy cập trong pandas.
Tên sau khi đổi tên:

['bra_size', 'bust', 'category', 'cup_size', 'fit', 'height', 'hips',
'item_id', 'length', 'quality', 'review_summary', 'review_text',
'shoe_size', 'shoe_width', 'size', 'user_id', 'user_name', 'waist']

In [ ]:
mc_df.columns

In [ ]:
## Rename the columns
# Bài tập 4: Kiểm tra và đổi tên cột
# Kiểm tra xem mc_df đã tồn tại chưa
if 'mc_df' in locals():
    print("Tên cột ban đầu:")
    print(mc_df.columns.tolist())

    # Thay thế khoảng trắng bằng dấu gạch dưới
    mc_df.columns = mc_df.columns.str.replace(' ', '_')

    print("\nTên cột sau khi đổi tên:")
    print(mc_df.columns.tolist())
else:
    print("DataFrame 'mc_df' chưa được tải. Vui lòng chạy lại ô nhập dữ liệu.")


In [ ]:
## Shown info of dataa
if 'mc_df' in locals():
    print("Thông tin DataFrame:")
    # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
    # Sử dụng phương thức .info() để xem thông tin tổng quan (số dòng, cột, kiểu dữ liệu, giá trị non-null)
    ...
else:
    print("DataFrame 'mc_df' chưa được tải.")

Chúng ta có thể mở rộng các quan sát của mình về dữ liệu bị thiếu và các kiểu dữ liệu ở đây:

* Trong số 18 cột, chỉ có 6 cột có dữ liệu đầy đủ.
* Khá nhiều dữ liệu dường như bị thiếu trong các cột `bust`, `shoe width`, `shoe size` và `waist`.
* Chúng ta có thể muốn đặc biệt xem xét các mặt hàng **có** `shoe size` và `shoe width` - đây có thể là giày!
* Rất nhiều cột có kiểu dữ liệu chuỗi (object), cần được phân tích cú pháp thành kiểu dữ liệu category (cũng hỗ trợ tiêu thụ bộ nhớ hiệu quả).
* Cột `waist` đáng ngạc nhiên lại có rất nhiều giá trị NULL - xét rằng hầu hết dữ liệu từ Modcloth đến từ 3 danh mục 'váy, áo và quần'.

### Xem xét tỷ lệ phần trăm giá trị bị thiếu trên mỗi cột

In [ ]:
### Calculate total misssing data and perc miss

# Bài tập 6: Tính toán và hiển thị dữ liệu bị thiếu
if 'mc_df' in locals():
    # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
    # Tính tổng số giá trị bị thiếu cho mỗi cột, sắp xếp giảm dần
    total_missing = ...
    # Tính tỷ lệ phần trăm giá trị bị thiếu cho mỗi cột, sắp xếp giảm dần
    percent_missing = ...
    # Ghép hai kết quả trên thành một DataFrame mới tên là missing_data
    # Đặt tên cho các cột là 'Tổng số thiếu' và 'Tỷ lệ % thiếu' (total_missing, percentile_missing)
    missing_data = ...
    print("Dữ liệu bị thiếu theo cột:")
    print(missing_data)
else:
    print("DataFrame 'mc_df' chưa được tải.")

### Statistical description of numerical variables

In [ ]:
## Statistical description


# Bài tập 7: Hiển thị mô tả thống kê cho các cột số
if 'mc_df' in locals():
    print("Mô tả thống kê các biến số:")
    # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
    # Sử dụng phương thức .describe() để xem mô tả thống kê
    # Chỉ bao gồm các cột có kiểu dữ liệu số (sử dụng include=[np.number])
    ...
else:
    print("DataFrame 'mc_df' chưa được tải.")

Một số quan sát quan trọng hơn ở đây, trước khi chúng ta đi sâu vào thực hiện các tác vụ tiền xử lý trên dữ liệu của mình:

* `bra_size`, `hips` có thể không cần phải là float - kiểu dữ liệu category?
* Hầu hết các cỡ giày là khoảng 5-9, nhưng cỡ giày tối đa là 38! (Điều này đáng ngạc nhiên vì trang web sử dụng cỡ giày của Anh - UK sizing.)
* `size` có giá trị tối thiểu là 0 và `size` tối đa khớp với cỡ giày tối đa.

Hãy trực quan hóa các đại lượng số trong bộ dữ liệu của chúng ta dưới dạng biểu đồ hộp (boxplots), để có cảm nhận tốt hơn về các giá trị ngoại lai.

<a id="5"></a>
## Biểu đồ hộp của các biến số

In [ ]:
# Bài tập 8: Vẽ biểu đồ hộp cho các biến số
if 'mc_df' in locals():
    # Chọn các cột số tiềm năng để vẽ boxplot
    num_cols = ['bra_size','hips','quality','shoe_size','size','waist']

    # Loại bỏ các cột không tồn tại hoặc không phải số khỏi danh sách num_cols
    valid_num_cols = ...

    if valid_num_cols:
        plt.figure(figsize=(18,9))
        # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
        # Vẽ biểu đồ hộp cho các cột trong valid_num_cols
        # Chỉ chọn các cột này từ mc_df, loại bỏ các cột rỗng hoàn toàn (dropna) và gọi .boxplot()
        mc_df[valid_num_cols].dropna(axis=1, how='all').boxplot()
        plt.title("Biểu đồ hộp các biến số trong bộ dữ liệu Modcloth", fontsize=20)
        plt.xticks(rotation=45) # Xoay nhãn trục x nếu cần
        plt.show()
    else:
        print("Không tìm thấy cột số hợp lệ nào để vẽ biểu đồ hộp.")
else:
    print("DataFrame 'mc_df' chưa được tải.")

<a id="6"></a>
## Xử lý các giá trị ngoại lai

* **shoe_size**:
  Chúng ta có thể thấy rõ rằng giá trị tối đa duy nhất của cỡ giày (38) là một giá trị ngoại lai và lý tưởng nhất là chúng ta nên loại bỏ hàng đó hoặc xử lý giá trị ngoại lai đó. Hãy xem xét mục nhập đó trong dữ liệu của chúng ta.

In [ ]:
## remove outliers
# Bài tập 9: Tìm và hiển thị hàng có giá trị shoe_size ngoại lai
if 'mc_df' in locals() and 'shoe_size' in mc_df.columns:
    # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
    # Tìm giá trị lớn nhất trong cột 'shoe_size'
    ...
    print(f"Giá trị shoe_size lớn nhất: {max_shoe_size}")

    # Lọc và hiển thị các hàng có 'shoe_size' bằng với giá trị lớn nhất tìm được
    outlier_rows = ...
    print(f"\nCác hàng có shoe_size = {max_shoe_size}:")
    print(outlier_rows)

    # Quyết định xử lý: (Tạm thời không cần viết code xử lý ở đây)
    # Ví dụ: thay thế bằng NaN hoặc xóa hàng
    # mc_df.loc[mc_df['shoe_size'] == max_shoe_size, 'shoe_size'] = np.nan
    # mc_df.drop(outlier_rows.index, inplace=True)

else:
    print("DataFrame 'mc_df' chưa được tải hoặc không có cột 'shoe_size'.")

> Chúng ta có thể thấy rằng mục nhập có vẻ hợp lệ, ngoại trừ cỡ giày - nó có thể bị khách hàng nhập sai hoặc đơn giản là nhiễu. Chúng ta sẽ nhập giá trị này là null (NaN) vào lúc này.

In [ ]:
## add null

# Bài tập 10: Thay thế giá trị shoe_size ngoại lai bằng NaN
if 'mc_df' in locals() and 'shoe_size' in mc_df.columns:
    max_shoe_size = 38 # Giá trị ngoại lai đã xác định
    if max_shoe_size in mc_df['shoe_size'].values:
        # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
        # Sử dụng .loc để chọn các hàng có shoe_size == max_shoe_size
        # và gán giá trị np.nan cho cột 'shoe_size' tại các hàng đó
        ...
        print(f"Đã thay thế giá trị shoe_size = {max_shoe_size} bằng NaN.")
    else:
        print(f"Không tìm thấy giá trị shoe_size = {max_shoe_size} để thay thế.")
else:
    print("DataFrame 'mc_df' chưa được tải hoặc không có cột 'shoe_size'.")

* **bra_size**:
  Chúng ta có thể xem xét 10 cỡ áo ngực hàng đầu (chúng ta có thể thấy rằng biểu đồ hộp hiển thị 2 giá trị là ngoại lai, theo IQR - Khoảng tứ phân vị).

In [ ]:
### sort values by bra size and get 10 items

# Bài tập 11: Khám phá phân phối và ngoại lai của bra_size
if 'mc_df' in locals() and 'bra_size' in mc_df.columns:
    # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
    # Đếm số lần xuất hiện của mỗi giá trị trong cột 'bra_size'
    bra_size_counts = ...
    print("Số lượng theo bra_size (10 giá trị phổ biến nhất):")
    # Hiển thị 10 giá trị đầu tiên của bra_size_counts
    print(bra_size_counts.head(10))

    # Tính Q1, Q3, IQR để xác định ngưỡng ngoại lai
    Q1 = ...# Tứ phân vị thứ nhất
    Q3 = ... # Tứ phân vị thứ ba
    IQR = Q3 - Q1 # Khoảng tứ phân vị
    upper_bound = Q3 + 1.5*IQR # Ngưỡng trên (Q3 + 1.5*IQR)
    lower_bound = Q1 - 1.5*IQR # Ngưỡng dưới (Q1 - 1.5*IQR)

    print(f"\nNgưỡng trên cho ngoại lai bra_size (Q3 + 1.5*IQR): {upper_bound}")
    print(f"Ngưỡng dưới cho ngoại lai bra_size (Q1 - 1.5*IQR): {lower_bound}")

    # Tìm các giá trị duy nhất có thể là ngoại lai
    potential_outliers = ...
    print(f"\nCác giá trị bra_size có thể là ngoại lai: {potential_outliers}")

    # Hiển thị 10 hàng có giá trị bra_size lớn nhất
    if not mc_df['bra_size'].isnull().all():
         # Sử dụng .nlargest() để lấy 10 hàng có bra_size lớn nhất
         largest_bra_sizes = ...
         print("\n10 hàng có bra_size lớn nhất:")
         # Chỉ hiển thị các cột 'bra_size', 'cup_size', 'size', 'category'
         print(largest_bra_sizes[['bra_size', 'cup_size', 'size', 'category']])
    else:
         print("\nCột 'bra_size' không có giá trị số hoặc rỗng.")

else:
    print("DataFrame 'mc_df' chưa được tải hoặc không có cột 'bra_size'.")


<a id="7"></a>
## Phân phối kết hợp của bra_size và size

Chúng ta có thể trực quan hóa sự phân phối của `bra_size` so với `size` (hai biến) để hiểu về các giá trị.

In [ ]:
# Bài tập 12: Vẽ biểu đồ phân tán (scatter plot) cho bra_size và size
if 'mc_df' in locals() and 'bra_size' in mc_df.columns and 'size' in mc_df.columns:
    plt.figure(figsize=(18,8))
    plt.xlabel("bra_size", fontsize=18)
    plt.ylabel("size", fontsize=18)
    plt.suptitle("Phân phối kết hợp của bra_size và size", fontsize= 20)
    # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
    # Sử dụng plt.scatter() để vẽ biểu đồ phân tán
    # Trục x là mc_df.bra_size, trục y là mc_df['size']
    # Đặt marker='o', alpha=0.2, color='blue'
    plt.scatter(mc_df.bra_size, mc_df['size'], marker='o', alpha=0.2, color='blue')
    plt.grid(True, linestyle='--', alpha=0.6) # Thêm lưới để dễ đọc
    plt.show()
else:
    print("DataFrame 'mc_df' chưa được tải hoặc thiếu cột 'bra_size'/'size'.")

> Chúng ta không thể thấy bất kỳ độ lệch đáng kể nào so với hành vi thông thường đối với `bra_size`, thực tế đối với tất cả các biến số khác cũng vậy - chúng ta có thể mong đợi các giá trị ngoại lai 'rõ ràng', từ biểu đồ hộp, sẽ hoạt động tương tự. Bây giờ, chúng ta sẽ chuyển sang tiền xử lý bộ dữ liệu để có các trực quan hóa phù hợp.
>
> <a id="8"></a>
> # Làm sạch & Tiền xử lý dữ liệu
>
> Hãy xử lý các biến và thay đổi kiểu dữ liệu (dtype) thành loại phù hợp cho mỗi cột. Trước tiên, chúng ta định nghĩa một hàm để tạo biểu đồ phân phối của các biến khác nhau. Đây là phân phối ban đầu của các đặc trưng.
>
> **Lưu ý: Các biểu đồ phân phối cuối cùng ở [bên dưới](#dist_plots).**
>
> <a id="9"></a>
> ### Phân phối ban đầu của các đặc trưng

In [ ]:
# Hàm vẽ biểu đồ phân phối cho một cột (Đã cung cấp sẵn)
def plot_dist(col, ax):
    if col in mc_df.columns and mc_df[col].notnull().any():
        data_to_plot = mc_df[col][mc_df[col].notnull()]
        if data_to_plot.nunique() > 50:
             top_values = data_to_plot.value_counts().nlargest(50)
             top_values.plot(kind='bar', facecolor='y', ax=ax)
             ax.set_title(f"Phân phối {col} (Top 50) trên Modcloth", fontsize= 16)
        else:
             data_to_plot.value_counts().plot(kind='bar', facecolor='y', ax=ax)
             ax.set_title(f"Phân phối {col} trên Modcloth", fontsize= 16)
        ax.set_xlabel(f'{col}', fontsize=18)
        ax.tick_params(axis='x', rotation=90)
    else:
        ax.set_title(f"Cột '{col}' không có dữ liệu hoặc không tồn tại", fontsize=16)
        ax.set_xlabel(f'{col}', fontsize=18)
    return ax

# Bài tập 13: Vẽ phân phối ban đầu của một số đặc trưng
if 'mc_df' in locals():
    # Danh sách các cột cần vẽ phân phối
    cols_to_plot = ['bra_size','bust', 'category', 'cup_size', 'fit', 'height', 'hips', 'length', 'quality']
    # Lọc ra các cột thực sự tồn tại trong DataFrame
    existing_cols = [col for col in cols_to_plot if col in mc_df.columns]

    if existing_cols:
        n_cols_exist = len(existing_cols)
        n_rows = (n_cols_exist + 2) // 3
        n_cols_subplot = 3
        # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
        # Tạo subplot với số hàng và cột đã tính, kích thước (22, 5 * n_rows)
        f, ax = plt.subplots(n_rows, n_cols_subplot, figsize=(22, 5 * n_rows))
        f.tight_layout(h_pad=9, w_pad=4, rect=[0, 0.03, 1, 0.95])
        ax = ax.flatten()

        k = 0
        # Sử dụng vòng lặp for để duyệt qua các subplot và gọi hàm plot_dist cho từng cột trong existing_cols
        for i in range(n_rows):
            for j in range(n_cols_subplot):
                if k < n_cols_exist:
                    # Gọi hàm plot_dist với cột và subplot tương ứng
                    plot_dist(existing_cols[k], ax[k])
                    k += 1
                else:
                     if k < len(ax):
                        ax[k].axis('off')
                     k += 1

        while k < len(ax):
            ax[k].axis('off')
            k += 1

        plt.suptitle("Phân phối ban đầu của các đặc trưng", fontsize= 25, y=1.02)
        plt.show()
    else:
        print("Không có cột nào trong danh sách `cols_to_plot` tồn tại trong DataFrame.")
else:
    print("DataFrame 'mc_df' chưa được tải.")

<a id="10"></a>
### Xử lý đặc trưng từng bước:

* **bra_size:** Mặc dù trông có vẻ là số, nó chỉ nằm trong khoảng từ 28 đến 48, với hầu hết các kích cỡ nằm trong khoảng 34-38. Việc chuyển đổi nó sang kiểu dữ liệu *category* là hợp lý. Chúng ta sẽ điền các giá trị NA vào một danh mục 'Unknown'. Chúng ta có thể thấy ở trên rằng hầu hết người mua có cỡ áo ngực là 34 hoặc 36.
* **bust**- Chúng ta có thể thấy bằng cách nhìn vào các giá trị không rỗng, rằng `bust` nên là kiểu dữ liệu số nguyên (integer). Chúng ta cũng cần xử lý một trường hợp đặc biệt khi `bust` được cung cấp là '37-39'. Chúng ta sẽ thay thế mục nhập '37-39' bằng giá trị trung bình, tức là 38, cho mục đích phân tích. Bây giờ chúng ta có thể chuyển đổi an toàn kiểu dữ liệu thành int. Tuy nhiên, xét rằng **khoảng 86% dữ liệu `bust` bị thiếu**, cuối cùng đã quyết định loại bỏ đặc trưng này.
* **category**- không có giá trị nào bị thiếu; thay đổi thành kiểu dữ liệu *category*.
* **cup size**- Thay đổi kiểu dữ liệu thành *category* cho cột này. Cột này có khoảng 7% giá trị bị thiếu. Xem xét các hàng mà giá trị này bị thiếu có thể gợi ý cho chúng ta cách xử lý các giá trị bị thiếu này.

In [ ]:
## fillna by Unknown and preprocessing as required


# Bài tập 14: Xử lý các cột bra_size, bust, category, cup_size
if 'mc_df' in locals():
    # Xử lý bra_size
    if 'bra_size' in mc_df.columns:
        # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
        # Điền giá trị NaN trong 'bra_size' bằng 'Unknown'
        # Chuyển đổi kiểu dữ liệu của cột 'bra_size' thành 'category'
        mc_df['bra_size'] = ...
        print("Đã xử lý 'bra_size'.")

    # Xử lý bust (loại bỏ)
    if 'bust' in mc_df.columns:
        # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
        # Sử dụng .drop() để loại bỏ cột 'bust' (axis=1)
        # Đặt inplace=True để thay đổi trực tiếp DataFrame
        ...
        print("Đã loại bỏ cột 'bust'.")

    # Xử lý category
    if 'category' in mc_df.columns:
        # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
        # Chuyển đổi kiểu dữ liệu của cột 'category' thành 'category'
        mc_df['category'] = ...
        print("Đã xử lý 'category'.")

    # Xử lý cup_size
    if 'cup_size' in mc_df.columns:
        print("\nCác giá trị duy nhất trong 'cup_size' trước khi xử lý:")
        print(mc_df['cup_size'].unique()[:20])
        # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
        # Điền giá trị NaN trong 'cup_size' bằng 'Unknown'
        # Chuyển đổi kiểu dữ liệu của cột 'cup_size' thành 'category'
        mc_df['cup_size'] = ...
        print("\nĐã xử lý 'cup_size'.")
        print("Các giá trị duy nhất trong 'cup_size' sau khi xử lý:")
        print(mc_df['cup_size'].unique())
else:
    print("DataFrame 'mc_df' chưa được tải.")


In [ ]:
# Bài tập 15: Hiển thị các mẫu có cup_size là 'Unknown'
if 'mc_df' in locals() and 'cup_size' in mc_df.columns and 'Unknown' in mc_df['cup_size'].cat.categories:
     print("\n20 mẫu ngẫu nhiên có cup_size là 'Unknown' (sau khi xử lý):")
     # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
     # Lọc ra các hàng có cup_size == 'Unknown'
     unknown_cup_size_df = ...
     # Lấy mẫu ngẫu nhiên (sample) 20 hàng từ kết quả lọc
     print(unknown_cup_size_df.sample(min(20, len(unknown_cup_size_df))))
elif 'mc_df' in locals() and 'cup_size' in mc_df.columns:
     print("\nKhông tìm thấy giá trị 'Unknown' trong cột 'cup_size' đã xử lý.")
else:
     print("DataFrame 'mc_df' chưa được tải hoặc không có cột 'cup_size'.")

In [ ]:
mc_df[mc_df.cup_size.isnull()].sample(20)

> Chúng ta không thể thấy bất cứ điều gì rõ ràng từ các hàng mà dữ liệu này bị thiếu, tuy nhiên, theo người quản lý bộ dữ liệu - "***Lưu ý rằng các bộ dữ liệu này rất thưa thớt, với hầu hết các sản phẩm và khách hàng chỉ có một giao dịch duy nhất.***" Điều đó chỉ ra rằng có lẽ những khách hàng này chưa mua đồ lót từ modcloth và do đó modcloth không có dữ liệu đó. Vì vậy, việc điền các giá trị null này là 'Unknown' là hợp lý. Từ sự phổ biến của các giá trị như dd/e, ddd/f và dddd/g, chúng ta có thể giả định đây là các cỡ cúp hợp lệ, cũng được xác nhận bởi bài viết [**này**](https://www.herroom.com/full-figure-bra-cup-sizing,905,30.html), trong đó một số thương hiệu thay đổi cỡ cúp dd thành e, ddd thành f, v.v. Chúng ta có thể chuyển đổi trực tiếp thành kiểu dữ liệu *category*.

* **fit**- Thay đổi kiểu dữ liệu thành *category* cho cột này. Chúng ta có thể thấy rằng đại đa số khách hàng đã đưa ra phản hồi 'vừa vặn' (fit) tốt cho các mặt hàng trên Modcloth!

In [ ]:
# Bài tập 16: Xử lý cột 'fit'
if 'mc_df' in locals():
    if 'fit' in mc_df.columns:
        print("Các giá trị duy nhất trong 'fit':", mc_df['fit'].unique())
        # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
        # Chuyển đổi kiểu dữ liệu của cột 'fit' thành 'category'
        mc_df['fit'] = ...
        print("Đã xử lý 'fit'.")
        print("\nSố lượng theo 'fit':")
        # In ra số lượng của mỗi giá trị trong cột 'fit'
        print(...)
else:
    print("DataFrame 'mc_df' chưa được tải.")

* **height**- Chúng ta cần phân tích cú pháp cột chiều cao vì hiện tại nó là một đối tượng chuỗi, có dạng - Xft. Yin. Việc chuyển đổi chiều cao sang cm sẽ hợp lý. Chúng ta cũng xem xét các hàng mà dữ liệu chiều cao bị thiếu.

In [ ]:
import re # Import thư viện regular expression

# Bài tập 17: Viết hàm chuyển đổi chiều cao và áp dụng
# Hàm chuyển đổi chiều cao từ 'X ft Y in' hoặc 'X\'Y"' sang cm
def get_cms(x):
    if pd.isnull(x):
        return np.nan
    try:
        # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
        # Sử dụng re.match để tìm feet và inches theo định dạng "(\d+)\s*'\s*(\d+)\s*\""
        match = ...
        if match:
            # Lấy giá trị feet và inches từ match.group() và chuyển thành số nguyên
            feet = ...
            inches = ...
            # Tính toán và trả về chiều cao bằng cm (1 ft = 30.48 cm, 1 in = 2.54 cm)
            return (feet * 30.48) + (inches * 2.54)
        else:
             return np.nan # Trả về NaN nếu không khớp định dạng
    except:
        return np.nan

if 'mc_df' in locals() and 'height' in mc_df.columns:
    print("Kiểu dữ liệu 'height' trước khi chuyển đổi:", mc_df['height'].dtype)
    print("Một số giá trị 'height' ban đầu:", mc_df['height'].unique()[:10])

    # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
    # Áp dụng hàm get_cms cho cột 'height' sử dụng .apply()
    mc_df['height'] = ...

    print("\nKiểu dữ liệu 'height' sau khi chuyển đổi:", mc_df['height'].dtype)
    print("Một số giá trị 'height' sau khi chuyển đổi (cm):", mc_df['height'].unique()[:10])
    # In số lượng giá trị null trong cột 'height' sau khi chuyển đổi
    print(f"\nSố giá trị null trong 'height' sau khi chuyển đổi: { mc_df['height'].isnull().sum() }")

    # Mô tả cột height sau chuyển đổi
    print("\nMô tả cột 'height' sau khi chuyển đổi:")
    # Sử dụng .describe() cho cột 'height'
    print(mc_df['height'].describe())
else:
    print("DataFrame 'mc_df' chưa được tải hoặc không có cột 'height'.")

In [ ]:
# Bài tập 18: Hiển thị các hàng có 'height' bị thiếu
if 'mc_df' in locals() and 'height' in mc_df.columns:
    # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
    # Lọc ra các hàng có giá trị 'height' bị thiếu (isnull())
    missing_height_df = ...
    print(f"Số hàng có 'height' bị thiếu: {len(missing_height_df)}")
    print("\n20 hàng đầu có 'height' bị thiếu (sau khi xử lý):")
    cols_to_show = ['item_id', 'category', 'fit', 'height', 'bra_size', 'cup_size', 'hips', 'shoe_size', 'shoe_width', 'waist', 'user_id']
    existing_cols_to_show = [col for col in cols_to_show if col in mc_df.columns]
    # Hiển thị 20 hàng đầu của missing_height_df với các cột trong existing_cols_to_show
    print(missing_height_df[existing_cols_to_show].head(20))
else:
    print("DataFrame 'mc_df' chưa được tải hoặc không có cột 'height'.")

> Việc lọc này cho chúng ta những quan sát thú vị ở đây:
>
> 1. Một số khách hàng đã cung cấp dữ liệu `bra_size`, `cup_size`, trong khi tất cả các số đo khác đều trống - có thể là lần mua hàng đầu tiên tại Modcloth cho đồ lót!
> 2. Một số khách hàng đã cung cấp `shoe_size` và tất cả các số đo khác đều trống - có thể là lần mua hàng đầu tiên tại Modcloth cho giày!
>
> Điều đó dẫn chúng ta đến việc nói rằng có một số người mua lần đầu trong bộ dữ liệu, cũng được các tác giả của dữ liệu đề cập trong [1] - về sự thưa thớt của dữ liệu do 1 giao dịch! Ngoài ra, vì chúng ta không có dữ liệu về chiều cao của những khách hàng này, nên chỉ có lý khi để lại các giá trị bị thiếu trong cột và **có thể loại bỏ các hàng này để lập mô hình thống kê trong tương lai.** Chúng ta đã loại bỏ các hàng tương ứng. (Lưu ý: Mã trong notebook gốc không xóa các hàng này ở bước này, cần xác nhận lại logic)
>
> <a id="11"></a>
> # Kỹ thuật đặc trưng (Feature Engineering)
>
> ## Tạo một đặc trưng mới: first_time_user
>
> Dựa trên các quan sát của chúng ta ở trên, việc xác định các giao dịch thuộc về người dùng lần đầu là hợp lý. Chúng ta sử dụng logic sau để xác định các giao dịch như vậy:
>
> * Nếu `bra_size`/`cup_size` có giá trị và `height`, `hips`, `shoe_size`, `shoe_width` và `waist` không có - đó là người mua đồ lót lần đầu.
> * Nếu `shoe_size`/`shoe_width` có giá trị và `bra_size`, `cup_size`, `height`, `hips`, và `waist` không có - đó là người mua giày lần đầu.
> * Nếu `hips`/`waist` có giá trị và `bra_size`, `cup_size`, `height`, `shoe_size`, và `shoe_width` không có - đó là người mua váy/áo lần đầu.
>
> Dưới đây chúng ta sẽ xác minh logic trên, với các mẫu, trước khi chúng ta tạo đặc trưng mới.
>
> **1. Xem xét một vài hàng mà `bra_size` hoặc `cup_size` tồn tại, nhưng không có số đo nào khác.**
> **2. Xem xét một vài hàng mà `shoe_size` hoặc `shoe_width` tồn tại, nhưng không có số đo nào khác.**
> **3. Xem xét một vài hàng mà `hips` hoặc `waist` tồn tại, nhưng không có số đo nào khác.**

In [ ]:
## Do the following


# Bài tập 19: Xác định các điều kiện cho người dùng lần đầu
if 'mc_df' in locals():
    required_cols = ['bra_size', 'cup_size', 'height', 'hips', 'shoe_size', 'shoe_width', 'waist']
    if all(col in mc_df.columns for col in required_cols):

        # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
        # Điều kiện 1: Mua đồ lót lần đầu
        # (bra_size != 'Unknown' HOẶC cup_size != 'Unknown') VÀ
        # (height, hips, shoe_size, shoe_width, waist đều là null)
        lingerie_cond = ...
        print("--- Mẫu người mua đồ lót lần đầu ---")
        print(mc_df[lingerie_cond][required_cols + ['category']].head())

        # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
        # Điều kiện 2: Mua giày lần đầu
        # (bra_size == 'Unknown' VÀ cup_size == 'Unknown') VÀ
        # (height, hips, waist đều là null) VÀ
        # (shoe_size không null HOẶC shoe_width không null)
        # Lưu ý: Kiểm tra shoe_width không null cần phù hợp với kiểu dữ liệu (số hoặc category)
        shoe_width_notnull_check = ...
        shoe_cond = ...
        print("\n--- Mẫu người mua giày lần đầu ---")
        print(mc_df[shoe_cond][required_cols + ['category']].head())

        # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
        # Điều kiện 3: Mua váy/áo lần đầu
        # (bra_size == 'Unknown' VÀ cup_size == 'Unknown') VÀ
        # (height, shoe_size, shoe_width đều là null) VÀ
        # (hips không null HOẶC waist không null)
        # Lưu ý: Kiểm tra shoe_width null cần phù hợp với kiểu dữ liệu
        shoe_width_isnull_check = mc_df.shoe_width.isnull() if pd.api.types.is_numeric_dtype(mc_df.shoe_width) else mc_df.shoe_width == 'Unknown'
        dress_cond = ((mc_df.bra_size == 'Unknown') & (mc_df.cup_size == 'Unknown') &
                      mc_df.height.isnull() &
                      (mc_df.hips.notnull() | mc_df.waist.notnull()) &
                      mc_df.shoe_size.isnull() & shoe_width_isnull_check)
        print("\n--- Mẫu người mua váy/áo lần đầu ---")
        print(mc_df[dress_cond][required_cols + ['category']].head())

    else:
        print("Một số cột cần thiết để xác định người dùng lần đầu không tồn tại hoặc chưa được xử lý.")
else:
    print("DataFrame 'mc_df' chưa được tải.")

> Bây giờ chúng ta thêm một cột mới vào dữ liệu gốc - `first_time_user`, là một đặc trưng bool cho biết liệu người dùng của một giao dịch có phải là người dùng lần đầu hay không. Điều này dựa trên cơ sở rằng Modcloth không có thông tin trước đó về người đó, thực tế có thể người dùng mới đã thực hiện nhiều giao dịch trong lần đầu tiên!

In [ ]:
# Bài tập 20: Tạo cột 'first_time_user'
if 'mc_df' in locals():
    if all(col in mc_df.columns for col in required_cols):
        # Tái sử dụng các điều kiện từ bài tập trước (lingerie_cond, shoe_cond, dress_cond)
        # hoặc định nghĩa lại chúng ở đây nếu cần
        # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
        # lingerie_cond = ...
        # shoe_cond = ...
        # dress_cond = ...

        # In ra số lượng bản ghi khớp với từng điều kiện để kiểm tra
        print(f"Số lượng khớp điều kiện đồ lót: {lingerie_cond.sum()}")
        print(f"Số lượng khớp điều kiện giày: {shoe_cond.sum()}")
        print(f"Số lượng khớp điều kiện váy/áo: {dress_cond.sum()}")

        # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
        # Tạo cột mới 'first_time_user' bằng cách kết hợp 3 điều kiện bằng toán tử OR (|)
        mc_df['first_time_user'] = (lingerie_cond | shoe_cond | dress_cond)
        print("\nĐã thêm cột 'first_time_user'!")
        # In tổng số giao dịch của người dùng lần đầu (tổng của cột 'first_time_user')
        print(f"Tổng số giao dịch của người dùng lần đầu (theo logic trên): { mc_df.first_time_user.sum() }")

        # Tính số người dùng lần đầu duy nhất
        # Lọc các hàng first_time_user=True, chọn cột 'user_id' và lấy các giá trị duy nhất (.unique())
        first_time_user_ids = mc_df[mc_df['first_time_user']]['user_id'].unique()
        print(f"Tổng số người dùng lần đầu duy nhất: {len(first_time_user_ids)}")

        # Kiểm tra một vài hàng được đánh dấu là first_time_user
        print("\nVí dụ về các hàng được đánh dấu là first_time_user=True:")
        # Lọc các hàng first_time_user=True và hiển thị các cột trong required_cols + ['first_time_user']
        print(mc_df[mc_df['first_time_user']][required_cols + ['first_time_user']].head())
    else:
        print("Không thể tạo cột 'first_time_user' do thiếu cột cần thiết hoặc cột chưa được xử lý.")
else:
    print("DataFrame 'mc_df' chưa được tải.")

* **hips**-
  Cột `hips` có rất nhiều giá trị bị thiếu ~ 32.28%! Chúng ta biết dữ liệu này có thể bị thiếu vì Modcloth rất có thể chưa bao giờ nhận được dữ liệu này từ người dùng. Chúng ta không thể loại bỏ một phần dữ liệu đáng kể như vậy, vì vậy chúng ta cần một cách khác để xử lý đặc trưng này. Chúng ta sẽ phân loại dữ liệu (binning) - trên cơ sở các tứ phân vị. (Lưu ý: Mã gốc sử dụng các ngưỡng tùy chỉnh, không phải tứ phân vị).
* **length**- Chỉ có 35 hàng bị thiếu trong `length`, chúng ta sẽ xem xét chúng. Chúng ta thấy rằng rất có thể khách hàng đã không để lại phản hồi hoặc dữ liệu đã bị hỏng trong các hàng này. Tuy nhiên, chúng ta có thể suy luận (impute) các giá trị này bằng cách sử dụng các trường liên quan đến đánh giá (nếu chúng được điền!). Hoặc chúng ta cũng có thể đơn giản chọn loại bỏ các hàng này. Vì mục đích của phân tích này, chúng ta sẽ loại bỏ các hàng này.
* **quality**- Chỉ có 68 hàng bị thiếu trong `quality`, chúng ta đã xem xét chúng. Tương tự như `length`, khách hàng đã không để lại phản hồi hoặc dữ liệu đã bị hỏng trong các hàng này. Chúng ta sẽ loại bỏ các hàng này và chuyển đổi kiểu dữ liệu thành một biến thứ tự (category có thứ tự).

In [ ]:
from pandas.api.types import CategoricalDtype # Import để tạo kiểu category có thứ tự

# Bài tập 21: Xử lý các cột hips, length, quality
if 'mc_df' in locals():
    # Xử lý cột hips ==> Chuyển đổi thành các khoảng (bins)
    if 'hips' in mc_df.columns:
        print(f"\nSố giá trị null trong 'hips' trước khi xử lý: {mc_df.hips.isnull().sum()}")
        # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
        # Điền giá trị NaN trong cột 'hips' bằng -1.0
        mc_df.hips = ...
        bins = [-5, 0, 31, 37, 40, 44, 75]
        labels = ['Unknown','XS','S','M', 'L','XL']
        # Sử dụng pd.cut để chia cột 'hips' thành các bin dựa trên bins và labels đã cho
        # Đặt right=False
        mc_df.hips = ...
        print("Đã xử lý 'hips' (binning).")
        print("Phân phối 'hips' sau khi binning:")
        print(mc_df.hips.value_counts())

    # Xử lý cột length ==> loại bỏ các hàng bị thiếu
    if 'length' in mc_df.columns:
        initial_rows = len(mc_df)
        # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
        # Lấy index của các hàng có 'length' bị thiếu
        missing_length_rows = ...
        # Sử dụng .drop() để loại bỏ các hàng này (axis=0), inplace=True
        mc_df.drop(missing_length_rows, axis=0, inplace=True)
        print(f"\nĐã xử lý 'length': Loại bỏ {len(missing_length_rows)} hàng bị thiếu.")
        print(f"Số hàng còn lại: {len(mc_df)}")

    # Xử lý quality ==> loại bỏ các hàng bị thiếu và chuyển thành category có thứ tự
    if 'quality' in mc_df.columns:
        initial_rows = len(mc_df)
        # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
        # Lấy index của các hàng có 'quality' bị thiếu
        missing_quality_rows = ...
        # Sử dụng .drop() để loại bỏ các hàng này (axis=0), inplace=True
        mc_df.drop(missing_quality_rows, axis=0, inplace=True)
        print(f"\nĐã xử lý 'quality': Loại bỏ {len(missing_quality_rows)} hàng bị thiếu.")
        print(f"Số hàng còn lại: {len(mc_df)}")

        # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
        # Xác định thứ tự của các giá trị quality duy nhất (sử dụng sorted() và .unique())
        quality_order = ...
        # Tạo kiểu CategoricalDtype với categories=quality_order và ordered=True
        quality_cat_type = CategoricalDtype(categories=quality_order, ordered=True)
        # Áp dụng kiểu dữ liệu này cho cột 'quality' sử dụng .astype()
        mc_df['quality'] = mc_df['quality'].astype(quality_cat_type)
        print("Đã chuyển 'quality' thành kiểu category có thứ tự.")
        print("Phân phối 'quality':")
        print(mc_df.quality.value_counts())

else:
    print("DataFrame 'mc_df' chưa được tải.")

* **review_summary/ review_text**- Các giá trị NA tồn tại vì các đánh giá này đơn giản là không được khách hàng cung cấp. Chúng ta chỉ cần điền chúng là 'Unknown'.
* **shoe_size** - Khoảng 66.3% dữ liệu `shoe_size` bị thiếu. Chúng ta sẽ thay đổi `shoe_size` thành kiểu dữ liệu *category* và điền các giá trị NA là 'Unknown'.
* **shoe_width** - Khoảng 77.5% dữ liệu `shoe_width` bị thiếu. Chúng ta sẽ điền các giá trị NA là 'Unknown' và chuyển thành kiểu category có thứ tự.
* **waist**- Cột `waist` có số lượng giá trị bị thiếu cao nhất - 96.5%! Chúng ta sẽ loại bỏ cột này.
* **bust**- 85.6% giá trị bị thiếu và tương quan cao với `bra_size`. Loại bỏ. (Đã loại bỏ ở bước trước)
* **user_name**- `user_name` không cần thiết khi đã có `user_id`. Loại bỏ.

Để chuyển đổi `shoe_width` thành kiểu category có thứ tự, chúng ta phải import `CategoricalDtype` và cung cấp thứ tự của các category.

In [ ]:
# Bài tập 22: Xử lý các cột còn lại
if 'mc_df' in locals():
    # Xử lý review_summary/review_text
    if 'review_summary' in mc_df.columns:
        # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
        # Điền giá trị NaN trong 'review_summary' bằng 'Unknown'
        mc_df['review_summary'] = mc_df['review_summary'].fillna('Unknown')
        print("Đã xử lý 'review_summary'.")
    if 'review_text' in mc_df.columns:
        # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
        # Điền giá trị NaN trong 'review_text' bằng 'Unknown'
        mc_df['review_text'] = ...
        print("Đã xử lý 'review_text'.")

    # Xử lý shoe_size
    if 'shoe_size' in mc_df.columns:
        # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
        # Điền giá trị NaN trong 'shoe_size' bằng 'Unknown'
        # Chuyển đổi kiểu dữ liệu của 'shoe_size' thành 'category'
        mc_df['shoe_size'] = ...
        print("\nĐã xử lý 'shoe_size'.")
        print("Phân phối 'shoe_size' (Top 10):")
        print(mc_df['shoe_size'].value_counts().head(10))

    # Xử lý shoe_width
    if 'shoe_width' in mc_df.columns:
        print(f"\nSố giá trị null trong 'shoe_width' trước khi xử lý: {mc_df.shoe_width.isnull().sum()}")
        # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
        # Điền giá trị NaN trong 'shoe_width' bằng 'Unknown'
        mc_df['shoe_width'] = ...

        # Xác định thứ tự (ví dụ)
        unique_widths = [w for w in mc_df['shoe_width'].unique() if w != 'Unknown']
        width_order = ['Unknown']
        if 'narrow' in unique_widths: width_order.append('narrow')
        if 'average' in unique_widths: width_order.append('average')
        if 'wide' in unique_widths: width_order.append('wide')
        remaining_widths = [w for w in unique_widths if w not in width_order]
        width_order.extend(sorted(remaining_widths))

        print("Thứ tự giả định cho 'shoe_width':", width_order)
        try:
            # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
            # Tạo CategoricalDtype với categories=width_order, ordered=True
            shoe_width_cat_type = CategoricalDtype(categories=width_order, ordered=True)
            # Áp dụng kiểu dữ liệu này cho cột 'shoe_width'
            mc_df['shoe_width'] = mc_df['shoe_width'].astype(shoe_width_cat_type)
            print("Đã xử lý 'shoe_width' (thành category có thứ tự).")
        except ValueError as e:
             print(f"Lỗi khi chuyển 'shoe_width' thành category có thứ tự: {e}")
             print("Chuyển 'shoe_width' thành category thông thường.")
             mc_df['shoe_width'] = mc_df['shoe_width'].astype('category')

        print("Phân phối 'shoe_width':")
        print(mc_df['shoe_width'].value_counts())


    # Xử lý waist (loại bỏ)
    if 'waist' in mc_df.columns:
        # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
        # Loại bỏ cột 'waist' (axis=1, inplace=True)
        mc_df.drop('waist', axis=1, inplace=True)
        print("\nĐã loại bỏ cột 'waist'.")

    # Xử lý user_name (loại bỏ)
    if 'user_name' in mc_df.columns:
        # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
        # Loại bỏ cột 'user_name' (axis=1, inplace=True)
        mc_df.drop('user_name', axis=1, inplace=True)
        print("Đã loại bỏ cột 'user_name'.")

else:
    print("DataFrame 'mc_df' chưa được tải.")

In [ ]:
# Bài tập 23: Kiểm tra thông tin DataFrame sau khi xử lý
if 'mc_df' in locals():
    print("\nThông tin DataFrame sau khi làm sạch và tiền xử lý:")
    # <<< VIẾT MÃ CỦA BẠN VÀO ĐÂY >>>
    # Hiển thị thông tin DataFrame bằng .info()
    mc_df.info()

    # Kiểm tra lại các giá trị bị thiếu
    print("\nKiểm tra lại dữ liệu bị thiếu:")
    # Tính tổng số giá trị null cho mỗi cột
    print(mc_df.isnull().sum())
else:
    print("DataFrame 'mc_df' chưa được tải.")

We can see that now there are no more missing values! We can move onto visualizing and gaining more insight about the data.

<a id="12"></a>
# EDA via visualizations

<a id="13"></a>
<a id='dist_plots'></a>
## 1. Distribution of different features over Modcloth dataset

In [ ]:

# Vẽ biểu đồ density  của các cột trong data Modcloth
def plot_dist(col, ax):
    ...



cols = ['bra_size','category', 'cup_size', 'fit', 'height', 'hips', 'length', 'quality']
k = 0
for i in range(2):
    for j in range(4):
        plot_dist(cols[k], ax[i][j])
        k += 1
__ = plt.suptitle("Final Distributions of different features", fontsize= 23)

<a id="14"></a>
## 2. Categories vs. Fit/Length/Quality
Here, we will visualize how the items of different categories fared in terms of - fit, length, and quality. This will tell Modcloth which categories need more attention!

I have plotted 2 distributions in categories here:

**1. Unnormalized**- viewing the frequency counts directly- for comparison across categories. We also include the best fit, length, or quality measure in this plot.

**2. Normalized** -  viewing the distribution for the category after normalizing the counts, amongst the category itself- it will help us compare what are major reason for return amongst the category itself. We exclude the best sizing & quality measures, so as to focus on the pre-dominant reasons of return per category (if any).

In [ ]:

def plot_barh(df,col, cmap = None, stacked=False, norm = None):
    df.plot(kind='barh', colormap=cmap, stacked=stacked)
    fig = plt.gcf()
    fig.set_size_inches(24,12)
    plt.title("Category vs {}-feedback -  Modcloth {}".format(col, '(Normalized)' if norm else ''), fontsize= 20)
    plt.ylabel('Category', fontsize = 18)
    plot = plt.xlabel('Frequency', fontsize=18)

def norm_counts(t):
    norms = np.linalg.norm(t.fillna(0), axis=1)
    t_norm = t[0:0]
    for row, euc in zip(t.iterrows(), norms):
        t_norm.loc[row[0]] = list(map(lambda x: x/euc, list(row[1])))
    return t_norm

In [ ]:
mc_df.category.value_counts()

<a id='references'></a>
# References:
    [1] Rishabh Misra, Mengting Wan, Julian McAuley Decomposing Fit Semantics for Product Size Recommendation in Metric Spaces. RecSys, 2018.
    
<a id='assumptions'></a>
# Assumptions:
The data source has been assumed as following on the Modcloth dataset:
* item_id- from item.
* waist- from user input.
* size - from item.
* quality- from user input.
* cup size- from user input.
* hips- from user input.
* bra size- from user input.
* category- from item.
* bust- from user input.
* height- from user input
* user_name- from user input
* length - from user input
* fit- from user input
* user_id- from user.


*sonnv*